In [39]:
import flywheel
import pandas as pd
from datetime import datetime

fw = flywheel.Client()
collection_id = '6184053063bd671113926cc7'
project_id = '5c508d5fc2a4ad002d7628d8' # NACC
project = fw.get(project_id)
gear = 'antsct-aging-fw'
gear_fw = fw.lookup('gears/antsct-aging-fw/0.3.1_0.3.3')

## Identify sessions without ANTs-CT runs

In [14]:
sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)]

In [60]:
# Create a data dict:
data_dict = {'subject':[],'session':[],'run':[],'status':[], 'label': []}

# Iterate over sessions
for session in sessions:

    # Because we want information off the sessions's analyses, we need to reload
    # The container to make sure we have all the metadata.
    session = session.reload()
    sub_label = session.subject.label
    ses_label = session.label
    # Any analyses on this session will be stored as a list:
    analyses = session.analyses

    # If there are no analyses containers, we know that this gear was not run
    if len(analyses) == 0:
        run = 'False'
        status = 'NA'
        label = 'NA'
    else:
        # Loop through the analyses
        matches = [asys for asys in analyses if asys.gear_info.get('name') == gear]
        # If there are no matches, the gear didn't run
        if len(matches) == 0:
            run = 'False'
            status = 'NA'
            label = 'NA'
        # If there is one match, that's our target
        elif len(matches) == 1:
            run = 'True'
            status = matches[0].job.get('state')
            label = matches[0].label
        # If there are more than one matches (due to reruns), take the most recent run.
        # This behavior may be modified to whatever suits your needs
        else:
            last_run_date = max([asys.created for asys in matches])
            last_run_analysis = [asys for asys in matches if asys.created == last_run_date]

            # There should only be one exact match
            last_run_analysis = last_run_analysis[0]

            run = 'True'
            status = last_run_analysis.job.get('state')
            label = last_run_analysis.label

    # Populate our data dict - remember that each key in the data dict must be updated
    # So that the length of our lists stays the same
    data_dict['subject'].append(sub_label)
    data_dict['session'].append(ses_label)
    data_dict['run'].append(run)
    data_dict['status'].append(status)
    data_dict['label'].append(label)

# Now create a data frame
df = pd.DataFrame.from_dict(data_dict)
df.head()

,subject,session,run,status,label
0,105476x01,105476x01x20190306x3T,False,NA,NA
1,123572,123572x20190313x3T,True,complete,antsct_2021-05-13_WT_PVS
2,119493,119493x20190320x3T,False,NA,NA
3,123894,123894x20190327x3T,True,complete,antsct_2021-05-13_WT_PVS
4,123725,123725x20190327x3T,True,complete,antsct_2021-05-13_WT_PVS


In [62]:
# Append a timestamp to our csv name so it won't overwrite anything when we upload it to flywheel
time_fmt = '%m-%d-%Y_%H-%M-%S'
time_string = datetime.now().strftime(time_fmt)
csv_out = f'../run_reports/{gear}_RunReport_{time_string}.csv'

df.to_csv(csv_out,index=False)

collection.upload_file(csv_out)

## Run ANTs

In [21]:
def get_bids_nifti(acq):
    '''
        Returns a BIDS-format NIfTI T1 image from an acquisition.
    '''
    bids_niftis = [f for f in acq.files if ('info' in f.keys() and 'BIDS' in f.info.keys() and "nii" in f.name)]
    if len(bids_niftis) == 1:
        return(bids_niftis.pop())
    elif len(bids_niftis) > 1:
        return(bids_niftis)
    else:
        return(None)

def get_t1_file(sess):
    '''
        Function to pick the most desirable T1 file out of several in a session. Very, very FTDC-specific.
    '''
    
    t1_acq = []
    acqlist = sess.acquisitions()
    for acq in acqlist:
        if any(['T1' in f.classification['Measurement'] for f in acq.files \
            if 'Measurement' in f.classification.keys()]):
                t1_acq.append(acq)
    
    t1_file = None
    
    for acq in t1_acq:
        lab = acq.label.lower()
        if ("t1" in lab) and not ("nd" in lab):
            t1_file = get_bids_nifti(acq)
            return(t1_file)
      
    t1_file = get_bids_nifti(t1_acq.pop())
    return(t1_file)

In [49]:
# date for label
x = datetime.now()
datestr = '%s-%s-%s_%s:%s' % (x.year, x.month, x.day, x.hour, x.minute)

# get sessions that haven't been analyzed yet
to_run = list(df.loc[df.run == 'False']['session']) # list of session label strings
for ses_label in to_run:
    ses = project.sessions.find(f"label={ses_label}")[0]
    t1_file = get_t1_file(ses)
    acq = t1_file.parent.label
    asys_label = f'antsct-aging_{datestr}_WT_PVS'
    print(f"RUNNING {asys_label} ON {ses_label}'S {acq} ACQUISITION...")
    # Run the gear
    inputs = {'t1_anatomy': t1_file}
    config = {'denoise': True, 'num-threads': 0, 'trim-neck-mode': 'mask', 'run-quick': False}
    # analysis_id = gear_fw.run(analysis_label=asys_label, config=config, inputs=inputs, destination=ses)


RUNNING antsct-aging_2021-11-5_12:50_WT_PVS ON 105476x01x20190306x3T'S T1_3D_0.8x0.8x0.8 ACQUISITION...
RUNNING antsct-aging_2021-11-5_12:50_WT_PVS ON 119493x20190320x3T'S T1_3D_0.8x0.8x0.8 ACQUISITION...
RUNNING antsct-aging_2021-11-5_12:50_WT_PVS ON 123434x20190402x3T'S T1_3D_0.8x0.8x0.8 ACQUISITION...
RUNNING antsct-aging_2021-11-5_12:50_WT_PVS ON 112574x20190403x3T'S T1_3D_0.8x0.8x0.8 ACQUISITION...
RUNNING antsct-aging_2021-11-5_12:50_WT_PVS ON 123950x20190403x3T'S T1_3D_0.8x0.8x0.8 ACQUISITION...
RUNNING antsct-aging_2021-11-5_12:50_WT_PVS ON 111275x01x20190416x3T'S T1_3D_0.8x0.8x0.8 ACQUISITION...
RUNNING antsct-aging_2021-11-5_12:50_WT_PVS ON 104115x20190507x3T'S T1_3D_0.8x0.8x0.8 ACQUISITION...
RUNNING antsct-aging_2021-11-5_12:50_WT_PVS ON 123072x20190508x3T'S T1_3D_0.8x0.8x0.8 ACQUISITION...
RUNNING antsct-aging_2021-11-5_12:50_WT_PVS ON 113066x02x20190528x3T'S T1_3D_0.8x0.8x0.8 ACQUISITION...
RUNNING antsct-aging_2021-11-5_12:50_WT_PVS ON 100113x20180821x3T'S T1_3D_0.8x0.8x

KeyboardInterrupt: 